# API Request 1

In [1]:
import pandas as pd 
import requests
from xml.etree import ElementTree as et

df = pd.read_excel("cities in counties.xlsx")
cities = list(df['City'])

df_cols = ["City", "Sname","Type", "gsId", "Lat", "Lon","Enrollment", "gsRating", "ParentRating"]
out_df = pd.DataFrame()


for  city in cities:
    url = ("https://api.greatschools.org/schools/CA/"
           +city+"/high-schools?key=02c6a5f6dff835ee51bdebc9000aaa44")
    r = requests.get(url)
    xml_data = r.content

    #print('got results for county ', county)
    
    #parse xml data
    root = et.fromstring(r.text)
    tree = et.ElementTree(root)
    tree.write("school2.xml")

    xtree = et.parse("school2.xml")
    xroot = xtree.getroot()
    
    rows = []

    for node in xroot: 
        name = node.find("name").text if node is not None else None
        type_ = node.find("type").text if node is not None else None
        gsId = node.find("gsId").text if node is not None else None
        lat = node.find("lat").text if node is not None else None
        lon = node.find("lon").text if node is not None else None
        enrollment = node.findtext('enrollment') if node is not None else None
        gsRating = node.findtext('gsRating') if node is not None else None
        parentRating = node.findtext('parentRating') if node is not None else None
        rows.append({"City": city, "Sname": name, "Type":type_,
                     "gsId": gsId, "Lat": lat, "Lon": lon,"Enrollment": enrollment,
                     "gsRating": gsRating, "ParentRating": parentRating})

    temp_df = pd.DataFrame(rows, columns = df_cols)
    
    out_df = out_df.append(temp_df, ignore_index = True)

In [2]:
r.status_code

200

In [4]:
out_df.head()

,City,Sname,Type,gsId,Lat,Lon,Enrollment,gsRating,ParentRating
0,Alameda,Alameda Community Learning Center,charter,11902,37.779053,-122.28703,360,8,4
1,Alameda,Alameda High School,public,1,37.764294,-122.24807,1853,9,4
2,Alameda,Alameda Science And Technology Institute,public,14052,37.779884,-122.28052,175,10,4
3,Alameda,"Applied Scholastics Academy, East Bay",private,17140,37.76968,-122.24931,13,None,5
4,Alameda,Children's Learning Center,private,10045,37.76858,-122.25329,None,None,5


In [5]:
out_df.shape

(4666, 9)

In [6]:
out_df = out_df.drop_duplicates(subset=['Lat','Lon'], keep='first')

In [7]:
out_df.shape

(4271, 9)

In [9]:
# exporting data
out_df.to_csv('school_gsid.csv')